In [5]:
import pandas as pd
import os

In [6]:
def treat_portion(x):
    x = x.replace(' [SEP] ', '').strip()
    token_list = x.split(' ')
    treat_cnt = 0
    if len(token_list) > 0:
        for token in token_list:
            if 'diag' not in token:
                treat_cnt += 1
    
    score = (treat_cnt + 1e-8)/ (len(token_list) + 1e-8)
    score = max(score, 0.001)
    score = min(score, 0.999)

    return score

DATA_PATH = '/nfs/turbo/lsa-regier/emr-data/'

groups = [str(i) for i in range(10)]

for group in groups:
    data_path = os.path.join(DATA_PATH, f'group_{group}_merged.csv')

    data = pd.read_csv(data_path, sep=',')
    data = data.assign(base_propensity_score = data['document'].apply(lambda x: treat_portion(x)))
    
    break